# Storage and Volumes 

State which graph driver should be used on which OS  

overlay2 is the preferred stroage driver for all supported Linux Distributions. 

devicemapper is supported, but requires direct-lvm for production environments.

aufs is the preferred storage driver for Docker 18.06 and older when running on Ubuntu 14.04 which has no support for overlay2 

btrfs and zfs storage drives are used if they are the backing filesystem 

vfs storage driver is intended for testing purposes

#### Demonstrate how to configure devicemapper

In [ ]:
vi /etc/docker/daemon.json 

In [ ]:
{
    "storage-driver": "devicemapper"
}

#### Compare object storage to block storage, and explain which one is preferable when available

In [ ]:
Block Storage 


Object Storage 

#### Summarize how an application is composed of layers and where those layers reside on the filesystem

A docker image is built up, from a series of layers, each representing a single instruction in the image's Dockerfile. Each layer except for the last, is a read-only layer. 

The configured Docker Storage Driver handles the details about how the image layers interact. With each container having its own writable container layer, each image can maintain a 1 to N ratio of image storage to container storage. As a result of Docker's storage strategy of 'Copy-On-Write', files are directories that exist in lower layers that are needed in higher layers, can be provided read access to them to avoid duplication. If that file needs to be modified, only then is it copied to the higher layer where the changes are stored. 


#### Describe how volumes are used with Docker for persistent storage

In [2]:
%%bash 
docker volume ls 
docker volume create my-mount

DRIVER              VOLUME NAME
local               5a3b287816651190790b52ddcf2340d212398131e7b561180c95293d3692b1ce
local               60c0aeb1f7c88f7a3889a7d4be0a50e4e941294e8a88dafb459eb95da6b185e8
local               719fa2c8aa4ee91de6c15528cd82fe3beb05154f681874b6333a9cc88c8f0413
local               845f1fa1f7f789d85f04d52984d56000f48cd829d66bf38ed2059134e44be210
local               05427482d5bdd725ed49ee75f49449a8613f6d7c9be210fc0008e3ab9cdfafa3
local               ad17604f9fbf88060c8f624f3d5fc44309c82fdf49947ca4681f655be81ff6a8
local               datadev
local               datavol
my-mount


In [5]:
%%bash 
docker volume ls | grep my-mount

local               my-mount


In [6]:
%%bash 
docker volume inspect my-mount | more

[
    {
        "CreatedAt": "2019-01-26T17:58:14Z",
        "Driver": "local",
        "Labels": {},
        "Mountpoint": "/var/lib/docker/volumes/my-mount/_data",
        "Name": "my-mount",
        "Options": {},
        "Scope": "local"
    }
]


In [19]:
%%bash 
sudo cd /var/lib/docker/volumes/my-mount/_data
sudo su 
echo "hello from the host" > /var/lib/docker/volumes/my-mount/_data/hostfile.txt 

In [13]:
%%bash 
docker service create --name testweb -p 80:80 --mount source=my-mount,target=/internal-mount --detach=false --replicas 3 httpd

o4zeo7bk30m5puca5f7v2ozdt
overall progress: 0 out of 3 tasks
1/3:  
2/3:  
3/3:  
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall progress: 0 out of 3 tasks
overall 

In [14]:
%%bash 
docker service ls 

ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
o4zeo7bk30m5        testweb             replicated          3/3                 httpd:latest        *:80->80/tcp


In [15]:
%%bash 
docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
74d30c57a3ab        httpd:latest        "httpd-foreground"       13 minutes ago      Up 13 minutes       80/tcp                   testweb.1.u9k6m60bxwxblq4ft92rrwhyj
b929a0167724        httpd:latest        "httpd-foreground"       13 minutes ago      Up 13 minutes       80/tcp                   testweb.3.pgqo1jjzcfnknqqz1rskldgac
4a7325baa870        httpd:latest        "httpd-foreground"       13 minutes ago      Up 13 minutes       80/tcp                   testweb.2.s3jwlmvr1lqe4454nrf8p46o5
219895035b09        registry:2          "/entrypoint.sh /etc…"   10 hours ago        Up 10 hours         0.0.0.0:5000->5000/tcp   elegant_benz


#### Identify the steps you would take to clean up unused images on a filesystem, also on DTR

#### Demonstrate how storage can be used across cluster nodes